In [1]:
import cv2
import dlib
import numpy as np

In [ ]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Improved function to detect stress from facial features
def detect_stress(landmarks):
    # Eyebrow height relative to eye position (for scrunching detection)
    left_eye_y = (landmarks.part(37).y + landmarks.part(38).y + landmarks.part(40).y + landmarks.part(41).y) / 4
    right_eye_y = (landmarks.part(43).y + landmarks.part(44).y + landmarks.part(46).y + landmarks.part(47).y) / 4
    
    left_brow_y = (landmarks.part(19).y + landmarks.part(20).y + landmarks.part(21).y) / 3
    right_brow_y = (landmarks.part(22).y + landmarks.part(23).y + landmarks.part(24).y) / 3
    
    # Distance between eyes and eyebrows (smaller when scrunched)
    left_distance = left_eye_y - left_brow_y
    right_distance = right_eye_y - right_brow_y
    
    # Measure inner eyebrow position (points 21 and 22) - gets closer when frowning
    inner_brow_distance = np.sqrt((landmarks.part(21).x - landmarks.part(22).x)**2 + 
                                  (landmarks.part(21).y - landmarks.part(22).y)**2)
    
    # Calculate nose bridge to chin distance for facial normalization
    face_height = landmarks.part(8).y - landmarks.part(27).y
    
    # Normalized metrics (to account for different face sizes and distances from camera)
    norm_inner_brow = inner_brow_distance / face_height
    norm_left_dist = left_distance / face_height
    norm_right_dist = right_distance / face_height
    
    # Multi-feature stress detection with adjusted thresholds
    stress_score = 0
    
    # Lower eyebrow-to-eye distance indicates stress
    if norm_left_dist < 0.11 or norm_right_dist < 0.11:  # Adjust these thresholds with testing
        stress_score += 1
        
    # Closer inner eyebrows indicate stress/frowning
    if norm_inner_brow < 0.15:  # Adjust threshold with testing
        stress_score += 1
    
    # Asymmetry in eyebrow height can indicate stress
    if abs(left_brow_y - right_brow_y) > 5:  # Less sensitive to tilting
        stress_score += 0.5
        
    return stress_score > 0.5, stress_score  # Return both detection and score

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    for face in faces:
        landmarks = landmark_predictor(gray, face)
        
        # Use improved stress detection
        stress_detected, stress_score = detect_stress(landmarks)

        # Draw result with score
        text = f"Stress: {stress_score:.1f}" if stress_detected else f"Relaxed: {stress_score:.1f}"
        color = (0, 0, 255) if stress_detected else (0, 255, 0)  # Red for stress, green for relaxed
        cv2.putText(frame, text, (face.left(), face.top() - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # Draw ALL facial landmarks
        for i in range(68):
            pt = (landmarks.part(i).x, landmarks.part(i).y)
            cv2.circle(frame, pt, 1, (0, 255, 0), -1)
            
        # Highlight eyebrow landmarks specifically
        for i in range(19, 25):
            pt = (landmarks.part(i).x, landmarks.part(i).y)
            cv2.circle(frame, pt, 3, (0, 0, 255), -1)
            
        # Highlight eye landmarks for reference
        for i in range(36, 48):
            pt = (landmarks.part(i).x, landmarks.part(i).y)
            cv2.circle(frame, pt, 2, (255, 0, 0), -1)

    cv2.imshow('Stress Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()